# Functional Programming Techniques in Python: Part 4

This is the last section of Function Programming Techniques (FPT) in Python. 

In this section we'll cover suggestions for adoption and migration to FPT as well as cover a few 'gotchas' when using closures. 

In [1]:
import datetime
import operator as op

In [2]:
print("Today is {}".format(datetime.datetime.now()))

Today is 2019-02-26 20:31:43.291057


# General Suggestions for Migrating to FPT

- Don't start forcing a FPT heavy design on existing repos or packages. Specifically for cases where you might be changing the public API of core libs of your application or stack. For example, migrating to heavily use [toolz](https://toolz.readthedocs.io/en/latest/streaming-analytics.html) might create too many impedence mismatches (style wise) in your software stack.
- Hold brown bag sessions during lunch (or similar) to help your team/organization understand or aren't fluent with the FPT patterns approach.
- Expanding your knowledge to FPT will help you understand and extend other libs, such as `click`. Having a solid understanding of FPT will provide you with one more tool in help you express yourself in code.
- Find specific examples in your application or code base where you're having composiblity issues with OO patterns and investigate using a FPT design. Be sure to discuss composibilty improvements as well as (hopefully) reduced cognative overhead of the code base.
- In FPT you're often going to have a lot of small functions, **consistent naming conventions** are extremely important. Perhaps even more so than in OO design. ([Obligatory naming is hard discussion](https://hilton.org.uk/blog/why-naming-things-is-hard))
- If you're using Python 3. Try to consistently leverage type declarations when passing around functions as arguments to other functions. 

*As with any pattern, it's essential to judiciously use the a pattern and not treat the pattern as a hammer to every problem you run across.*

Here's a few misc comments or items before we close out the FPT series.

# A Few Misc Comments

## Scope Gotchas

In Python 2.x, I found Python's sense of scope to be a bit odd at times. Python 3 added `nonlocal` via PEP-3104. The detailed discussion is here and is an interesting read.

https://www.python.org/dev/peps/pep-3104/

To demonstrate potential scope *gotchas*, here an example in Python 3 using a simple caching mechanism, slightly inspired from Python's [lru_cache](https://docs.python.org/3/library/functools.html#functools.lru_cache)

### Example #1

In [3]:
def compute(f=op.add):
    
    cache = {}

    def f2(m, n):
        key = (m, n)
        if key in cache:
            value = cache[key]
            print("Loading from cache {}={}".format(key, value))
            return value
        else:
            v = f(m, n)
            cache[key] = v
            return v
    return f2

In [4]:
f = compute()

In [5]:
f(1, 2)

3

In [6]:
for _ in range(2):
    f(1, 2)

Loading from cache (1, 2)=3
Loading from cache (1, 2)=3


In [7]:
f(1, 3)

4

## Example #2

Let's modify the previous example by only keeping track of how many times the function is called using the same pattern.

This will not work as expected and will raise an `UnboundLocalError`. 

In [8]:
def compute(f=op.add):
    
    num_times_called = 0
    
    def get_num_times():
        """Return the cache size"""
        return num_times_called

    def f2(m, n):
        # In scoping rules in Python num_times_called is never defined
        num_times_called += 1
        return f(m, n)


    f2.get_num_times = get_num_times
    return f2

In [9]:
f = compute()

In [10]:
f(2, 3)

UnboundLocalError: local variable 'num_times_called' referenced before assignment

You can solve this in Python 3 by using `nonlocal`. 

https://docs.python.org/3/reference/simple_stmts.html#nonlocal

In [11]:
def compute(f=op.add):
    
    num_times_called = 0
    
    def get_num_times():
        """Return the cache size"""
        return num_times_called

    def f2(m, n):
        nonlocal num_times_called
        
        num_times_called += 1
        return f(m, n)

    f2.get_num_times = get_num_times
    return f2

In [12]:
f = compute()
f(1, 2)

3

In [13]:
f(2, 3)

5

In [14]:
f.get_num_times()

2

A workaround/hack for Python 2 is to use a container object, such as a list.

In [15]:
def two_seven(n):
    # DONT do this in Python 3. Use nonlocal.
    total = [0]
    
    def f(m):
        total[0] += 1
        return n + m
    
    def get_total():
        return total[0]
    
    # see comments below about use of this pattern
    f.get_total = get_total
    return f

def two_seven_example():
    f27 = two_seven(2)
    def fx():
        return f27(1)
    for _ in range(5):
        for name, func in (("calling func", fx), ("total times called", f27.get_total)):
            print("{}={}".format(name, func()))

two_seven_example()

calling func=3
total times called=1
calling func=3
total times called=2
calling func=3
total times called=3
calling func=3
total times called=4
calling func=3
total times called=5


## Abusing Mutation within a Closure

It's easy to get carried away with a flat 'no' design approach. This really is not a great idea. It's very difficult to reason and debug. 

In [16]:
def bad_example(m):
    
    alpha = 1
    beta = 2
    gamma = 3.0
    
    def to_gamma(n):
        nonlocal gamma 
        gamma = (alpha + 1) * (beta + 2) + (n + m)
        
    def to_alpha(n):
        nonlocal alpha 
        alpha = beta * (n + m)
        
    def to_beta(n):
        nonlocal beta 
        beta = alpha * 3.14 * m
    
    def f(n):
        to_gamma(n)
        to_alpha(n)
        to_beta(n)
        return "alpha={:.2f}, beta={:.2f}, gamma={:.2f}".format(alpha, beta, gamma)
    
    return f

In [17]:
f = bad_example(7)

In [18]:
f(2)

'alpha=18.00, beta=395.64, gamma=17.00'

In [19]:
f(3)

'alpha=3956.40, beta=86961.67, gamma=7565.16'

This is analogous to a hyper mutable class with too many mutable instance variables. This is marginally different than using global variables and this should be avoided if possible. It's often better to split up the problem into smaller pieces and potentially migrate to an OO pattern.

A general guideline when designing with closures is it's often useful if the **closed over value is immutable or is explicitly a mutable variable used as a cache.**

(Also, the closure based approach is arguable worse because it's not easy to test the inner functions)

## Python Decorators

This is perhaps an omission from Part 1 of FPT. However, there are several terrific resources for this specific topic.

https://realpython.com/primer-on-python-decorators/

If you're not using Python decorators (specifically in designing your core libs or frameworks), you should probably invest a few cycles in understanding this core feature of Python. Decorators are also complementary to many of the FPT listed in Part 1.

Here's a small hello-world timer example. Please see [realpython.com](https://realpython.com/primer-on-python-decorators/) for more examples.

Here's real world examples from [production code used by Pacific Biosciences](https://github.com/mpkocher/pbcommand/blob/master/pbcommand/pb_io/tool_contract_io.py#L195).

In [20]:
import time

def custom_timer(f):
    def g(*args, **kw):
        started_at = datetime.datetime.now()
        result = f(*args, **kw)
        completed_at = datetime.datetime.now()
        dt = completed_at - started_at 
        msg = "Ran func {} at {} in {:.2f} sec with args={} kw={}".format(f.__name__, started_at, dt.total_seconds(), args, kw)
        print(msg)
        return result
    return g

@custom_timer    
def adder(a, b):
    time.sleep(0.25)
    return a + b

def run_example():
    nums = zip(range(1, 5), range(7, 11))
    for a, b in nums:
        out = adder(a, b)

In [21]:
run_example()

Ran func adder at 2019-02-26 20:32:08.086756 in 0.25 sec with args=(1, 7) kw={}
Ran func adder at 2019-02-26 20:32:08.337394 in 0.25 sec with args=(2, 8) kw={}
Ran func adder at 2019-02-26 20:32:08.591756 in 0.25 sec with args=(3, 9) kw={}
Ran func adder at 2019-02-26 20:32:08.846650 in 0.25 sec with args=(4, 10) kw={}


# Summary and Final Comments

- Make sure your team is fluent with leveraging
- Judicously apply FPT patterns when desiging your program or application
- Leverage `nonlocal` to avoid mutable scope issues
- Avoid too much mutation in inner functions. 
- Leverage decorators

If you're a data scientist, backend dev or even an OO-wizard, I hope you've picked up some tricks to better express your ideas in code using the **Functional Progamming Techniques (FPT) in Python series**. 

Best to you and your Python-ing!

In [22]:
print("Today is {}".format(datetime.datetime.now()))

Today is 2019-02-26 20:32:10.684615
